In [1]:
# %pip install requests
# %pip install beautifulsoup4
# %pip install pandas
# %pip install tqdm
# %pip install plotly
# %pip install statsmodels
# %pip install nbformat

In [2]:
import requests
from bs4 import BeautifulSoup
from itertools import islice
import warnings
import pandas as pd
from tqdm import tqdm
import plotly.express as px
import numpy as np
import statsmodels.api as sm
import plotly.graph_objects as go
warnings.filterwarnings("ignore")

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_England_Test_cricketers'
response = requests.get(url, verify=False)

In [4]:
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
# Step 1: Extract cricketer names and their reference IDs from the table rows
cricketers = soup.select('table.wikitable tr')
cricketer_info = []

for row in cricketers[1:]:  # Skip header row
    name_cell = row.find('th', scope='row')
    if name_cell:
        name = name_cell.text.strip()
        refs = row.find_all('sup', class_='reference')
        if refs:
            ref_ids = [ref.find('a')['href'] for ref in refs]
            cricketer_info.append((name, ref_ids))

# Step 2: Map reference IDs to Cricinfo URLs
cricinfo_links = {}

for name, ref_ids in cricketer_info:
    for ref_id in ref_ids:
        citation_link_id = ref_id.lstrip('#')
        citation = soup.find('li', id=citation_link_id)
        if citation:
            cricinfo_link = citation.find('a', class_='external text', href=True)
            if cricinfo_link and 'espncricinfo.com' in cricinfo_link['href']:
                cricinfo_links[name] = cricinfo_link['href']
                break  # Assuming only one relevant link per cricketer, break after finding the first

In [6]:
# Print the results
for name, link in islice(cricinfo_links.items(), len(cricinfo_links)-10, None):
    print(f"Name: {name}, Cricinfo Link: {link}")

Name: Matthew Potts, Cricinfo Link: http://www.espncricinfo.com/england/content/player/1027781.html
Name: Matt Parkinson, Cricinfo Link: http://www.espncricinfo.com/england/content/player/653695.html
Name: Jamie Overton, Cricinfo Link: http://www.espncricinfo.com/england/content/player/510530.html
Name: Harry Brook, Cricinfo Link: http://www.espncricinfo.com/england/content/player/911707.html
Name: Will Jacks, Cricinfo Link: http://www.espncricinfo.com/england/content/player/897549.html
Name: Liam Livingstone, Cricinfo Link: http://www.espncricinfo.com/england/content/player/403902.html
Name: Rehan Ahmed, Cricinfo Link: http://www.espncricinfo.com/england/content/player/1263691.html
Name: Josh Tongue, Cricinfo Link: https://www.espncricinfo.com/cricketers/josh-tongue-857975
Name: Tom Hartley, Cricinfo Link: https://www.espncricinfo.com/cricketers/tom-hartley-1150772
Name: Shoaib Bashir, Cricinfo Link: https://www.espncricinfo.com/cricketers/shoaib-bashir-1334872


In [7]:
def extract_df(html_content):

    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the "Batting & Fielding" section by looking for a p tag with specific text
    batting_fielding_section = soup.find('p', text=lambda x: x and 'Batting & Fielding' in x)

    # Assuming the batting table is immediately after the batting & fielding section
    batting_table = batting_fielding_section.find_next_sibling('div').find('table')

    df = pd.read_html(str(batting_table))[0]
    
    return df

In [8]:
# Example loop through cricinfo_links, assuming you have a way to fetch HTML content from URLs
player_data = []
for player_name, player_url in tqdm(islice(cricinfo_links.items(), len(cricinfo_links)-100, None)):
    html_content = requests.get(player_url, verify=False).text
    df_player = extract_df(html_content)
    df_player['Name'] = player_name
    player_data.append(df_player)

100it [01:52,  1.13s/it]


In [9]:
# Create a DataFrame from the player data
df = pd.concat(player_data)

In [10]:
df.head()

,Format,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100s,50s,4s,6s,Ct,St,Name
0,Tests,4,5,0,201,81,40.2,402,50.00,0,2,27,1,3,0,Anthony McGrath
1,ODIs,14,12,2,166,52,16.6,353,47.02,0,1,11,0,4,0,Anthony McGrath
2,FC,257,429,30,14698,211,36.83,-,-,35,70,-,-,181,0,Anthony McGrath
3,List A,296,272,41,7574,148,32.78,-,-,7,45,-,-,97,0,Anthony McGrath
4,T20s,66,61,12,1403,73*,28.63,1226,114.43,0,8,109,34,26,0,Anthony McGrath


In [11]:
df_tests = df[df['Format'] == 'Tests']
df_FC = df[df['Format'] == 'FC']
df_LA = df[df['Format'] == 'List A']

In [12]:
df_merge = (df_tests[['Name', 'Inns', 'NO', 'Runs', 'Ave']].merge(df_FC[['Name', 'Inns', 'NO', 'Runs', 'Ave']], on='Name', how='left', suffixes=['_test', ''])).\
    merge(df_LA[['Name', 'Inns', 'NO', 'Runs', 'Ave']], on='Name', how='left', suffixes=['_FC', '_LA'])

In [13]:
# Specify the filename
filename = 'cricketers.csv'

# Save the DataFrame to a CSV file
df_merge.to_csv(filename, index=False)

print(f'Data saved to {filename}')

Data saved to cricketers.csv


In [14]:
for col in df_merge.columns[1:]:
    df_merge[col] = pd.to_numeric(df_merge[col], errors='coerce')

In [15]:
df_merge['Ave_FC_no_test'] = ((df_merge['Runs_FC'] - df_merge['Runs_test']) / (df_merge['Inns_FC'] - df_merge['Inns_test'] - 
                                                                              (df_merge['NO_FC'] - df_merge['NO_test']))).round(2)

In [16]:
df_merge.head()

,Name,Inns_test,NO_test,Runs_test,Ave_test,Inns_FC,NO_FC,Runs_FC,Ave_FC,Inns_LA,NO_LA,Runs_LA,Ave_LA,Ave_FC_no_test
0,Anthony McGrath,5.0,0.0,201.0,40.20,429,30,14698,36.83,272,41,7574,32.78,36.79
1,Richard Johnson,4.0,0.0,59.0,14.75,227,28,3545,17.81,127,30,1108,11.42,17.88
2,James Kirtley,7.0,1.0,32.0,5.33,231,76,2040,13.16,94,50,447,10.15,13.48
3,Ed Smith,5.0,0.0,87.0,17.40,325,19,12789,41.79,131,9,3798,31.13,42.20
4,Kabir Ali,2.0,0.0,10.0,5.00,193,31,2755,17.00,108,28,1251,15.63,17.16


In [17]:
df_merge['Name'] = df_merge['Name'].replace('\xa0[\u2020\u2021]', '', regex=True)

In [18]:
df_filtered = df_merge[df_merge['Runs_test'] > 500]

In [19]:
df_filtered = df_filtered.dropna(subset=['Ave_LA', 'Ave_test'])

In [20]:
df_filtered['Ave_LA'].corr(df_filtered['Ave_test'])

0.481313705033907

In [21]:
df_filtered['Ave_FC_no_test'].corr(df_filtered['Ave_test'])

0.4811676452089918

In [22]:
df_filtered = df_filtered.rename(columns={'Ave_LA': 'List A Batting Average', 'Ave_test': 'Test Batting Average'})

In [23]:
# Assuming 'Test Batting Average' and 'List A Batting Average' are already numeric
X = df_filtered['List A Batting Average'].values.reshape(-1, 1)  # Predictor
y = df_filtered['Test Batting Average']  # Response

# Add a constant for the intercept to X
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y, X).fit()

# Predictions for the extended range
# Assume you want to extend the x-axis to cover List A averages up to 70
x_pred = np.linspace(0, 70, 100)  # Generate values from min to 70
x_pred2 = sm.add_constant(x_pred)  # Add a constant for the intercept
y_pred = model.predict(x_pred2)  # Make predictions

# Predict Sam Hain's Test Average
sam_hain_list_a_avg = 57.76
sam_hain_test_pred = model.predict([1, sam_hain_list_a_avg])[0]

# Create a DataFrame for Sam Hain's data
sam_hain_df = pd.DataFrame({
    'Name': ['Sam Hain'],
    'List A Batting Average': [sam_hain_list_a_avg],
    'Test Batting Average': [sam_hain_test_pred],
    'Label': ['Sam Hain']
})

# Concatenate Sam Hain's DataFrame with the original DataFrame
df_filtered = pd.concat([df_filtered, sam_hain_df], ignore_index=True)


# List of names you want to label
names_to_label = ['Kevin Pietersen', 'James Vince', 'Joe Root', 'Ollie Pope', 'Dan Lawrence', 'Gary Ballance', 'Ian Bell', 'Jonathan Trott', 'Dawid Malan']

# Create a new column for labels
df_filtered['Label'] = df_filtered['Name'].apply(lambda x: x if x in names_to_label else None)

# Create the scatter plot
fig = px.scatter(df_filtered, x='List A Batting Average', y='Test Batting Average',
                 text='Label',  # Use the new 'Label' column for text
                 hover_name='Name',  # Still use the original 'Name' column for hover text
                 title="Test Batting Average vs List A Batting Average")

# Optional: Update layout or traces if necessary
fig.update_traces(textposition='top center')

# Highlight Sam Hain in red
fig.add_trace(go.Scatter(x=[sam_hain_list_a_avg], y=[sam_hain_test_pred],
                         mode='markers+text', name='Sam Hain', text=['Sam Hain'],
                         textposition='top center', marker_color='red'))

# Add the extended trendline
fig.add_trace(go.Scatter(x=x_pred, y=y_pred, mode='lines', name='Trendline'))

# Update layout for x-axis to extend to 70
fig.update_layout(xaxis_range=[0, 70], yaxis_range=[0, 60])

# Show plot
fig.show()
